<a href="https://colab.research.google.com/github/prxrwx/Stat-learning-for-datasci/blob/main/Multivariate-linear-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# uploading the csv file to colab
!wget http://www.donlapark.cmustat.com/229351/data/Auto.csv

--2021-12-20 04:52:00--  http://www.donlapark.cmustat.com/229351/data/Auto.csv
Resolving www.donlapark.cmustat.com (www.donlapark.cmustat.com)... 150.107.31.67
Connecting to www.donlapark.cmustat.com (www.donlapark.cmustat.com)|150.107.31.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18079 (18K) [text/csv]
Saving to: ‘Auto.csv.1’

Auto.csv.1          100%[===================>]  17.66K  74.8KB/s    in 0.2s    

2021-12-20 04:52:01 (74.8 KB/s) - ‘Auto.csv.1’ saved [18079/18079]



In [ ]:
# import module
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# อ่านไฟล์ csv ก่อนเพื่อหา missing values
auto_df = pd.read_csv('Auto.csv',na_values=["?"])

# ลบแถวที่มี missing values
auto_df = auto_df.dropna()
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### ตัวแปรต่าง ๆ มีดังนี้
mpg

    miles per gallon
cylinders

    Number of cylinders between 4 and 8
displacement

    Engine displacement (cu. inches)
horsepower

    Engine horsepower
weight

    Vehicle weight (lbs.)
acceleration

    Time to accelerate from 0 to 60 mph (sec.)
year

    Model year (modulo 100)
origin

    Origin of car (1. American, 2. European, 3. Japanese)
name

    Vehicle name


In [ ]:
auto_df = auto_df.drop(['name', 'origin'], axis=1)

auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


## 1. Multivariate linear regression

เปลี่ยน $\texttt{dataframe}$ ให้เป็น numpy array ด้วย $\texttt{dataframe.to_numpy()}$  

1. Response: $\texttt{y}$ เป็นคอลัมน์เวกเตอร์ของ $\texttt{mpg}$
2. Predictors: $\texttt{X}$ เป็นเมทริกซ์ของตัวแปรที่เหลือ (ใช้ $\texttt{auto_df.drop}$)


In [ ]:
y = auto_df['mpg'].to_numpy()
X = auto_df.drop(['mpg'], axis=1).to_numpy()

คอลัมน์แรกของ $\texttt{X}$ ต้องเป็น $(1, 1, \ldots ,1 )^T$ (จำนวนเลข 1 เท่ากับจำนวนแถวของ $\texttt{X}$)  
จงสร้างเวกเตอร์ที่มีแต่เลข 1 และมีความยาวที่เหมาะสมด้วยคำสั่งใดคำสั่งหนึ่งต่อไปนี้  
1. $\texttt{[1]*len}$  
2. $\texttt{np.ones(len)}$  

In [ ]:
X.shape

(392, 6)

In [ ]:
ones =  np.ones(auto_df.shape[0])

# ใส่ ones ให้เป็นคอลัมน์แรกของ X
X = np.c_[ones, X]

In [ ]:
# เช็คคำตอบ
print(X)

[[1.000e+00 8.000e+00 3.070e+02 ... 3.504e+03 1.200e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.500e+02 ... 3.693e+03 1.150e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.180e+02 ... 3.436e+03 1.100e+01 7.000e+01]
 ...
 [1.000e+00 4.000e+00 1.350e+02 ... 2.295e+03 1.160e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.200e+02 ... 2.625e+03 1.860e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.190e+02 ... 2.720e+03 1.940e+01 8.200e+01]]


1. Matrix multiplication $AB = \texttt{A@B}$
2. Transpose: $X^T = \texttt{X.T}$
3. Inverse: $X^{-1} = \texttt{np.linalg.inv(X)}$

จงหาเวกเตอร์ของสัมประสิทธิ์
$$ \hat{\beta} = (X^TX)^{-1}X^Ty $$

In [ ]:
# Let's start by implementing linear regression from scratch
# using numpy linear algebra

def linear_model(X, y):   
    beta = (np.linalg.inv(X.T@X)) @ (X.T@y)
    return beta 

def predict(X, beta):
    return X@beta

In [ ]:
beta = linear_model(X, y)
y_pred = predict(X, beta)

In [ ]:
beta

array([-1.45352505e+01, -3.29859089e-01,  7.67843024e-03, -3.91355574e-04,
       -6.79461791e-03,  8.52732469e-02,  7.53367180e-01])

#### จงหา $n$ (จำนวนแถวของ $\texttt{X}$) และ $p$ (จำนวนตัวแปร) ด้วย method $\texttt{X.shape[...]}$

In [ ]:
n = X.shape[0]

p = X.shape[1]-1

print('Number of observations: ',n)
print('Number of variables: ',p)

Number of observations:  392
Number of variables:  6


## 2. $F$-statistic

####คำนวณ Total Sum of Squares (TSS)
$$ \text{TSS} = \sum_{i=1}^n (y_i-\bar{y})^2 $$


In [ ]:
# Calculate Total Sum of Squares
y_mean = y.mean()

TSS =  np.sum(np.square(y-y.mean()))

RSS = np.sum(np.square(y-y_pred))

คำนวณ $F$-statistic ด้วย
$$F_{p,n-p-1} = \frac{(\text{TSS}-\text{RSS})/p}{\text{RSS}/(n-p-1)}$$
หา $n$ และจำนวนของตัวแปร $p$ ด้วย $\texttt{X.shape}[\cdot]$

In [ ]:
F = ((TSS-RSS)/p) / (RSS/(n-p-1))

print(F)

272.23409822574575


In [ ]:
#Compute p_value of the F-statistic

p_value = stats.f.sf(F, p, n-p-1)

print(p_value)

3.792336100722869e-135


####คำถาม : จากค่า p-value ที่ได้ข้างบน เราสามารถสรุปอะไรได้




H0: $\beta_1$ = $\beta_2$ = ... = $\beta_6$

H1: มี $\beta_j$ อย่างน้อย 1 ตัว ไม่เท่ากับ 0 , j = 1, 2, 3, ..., 6

จาก p-value = 0.000 < 0.05 ตกในอาณาเขตวิกฤต Reject H0 นั่นคือ ตัวแบบที่ได้มีความเหมาะสม ที่ระดับนัยสำคัญ 0.05

## 3. คำนวณ $t$-statistic
$$  \text{RSE} = \sqrt{\frac{\text{RSS}}{n-p-1}} $$

In [ ]:
# Calculate Residual Standard Error
RSE = np.sqrt(RSS/(n-p-1))

####วิธีการคำนวณ Standard Error (SE)  
\begin{align*} \text{SE}(\hat{\beta}) &=\text{RSE}\cdot\sqrt{\text{diag}\left( (X^TX)^{-1}\right)} \\
t &= \frac{\hat{\beta}}{\text{SE}(\hat{\beta})}
\end{align*}
จงใช้ฟังก์ชัน $\texttt{t_to_p_values}$ ข้างล่างในการหาว่ามีตัวแปรใดบ้างที่มีค่า p-value น้อยกว่า 0.05 **แล้วตอบลงใน text block ข้างล่าง**  

ใช้คำสั่ง $\texttt{A.diagonal()}$ เพื่อหาสมาชิกในแนวทแยงของ $\texttt{A}$

In [ ]:
inv = np.linalg.inv(X.T@X)
SE_beta = RSE*np.sqrt(inv.diagonal())

In [ ]:
def t_to_p_values(t_statistics):
  return stats.t.sf(np.abs(t_statistics), n-p-1)*2
# Standard error of each variable in X

t = beta/SE_beta

p = t_to_p_values(t)
p

array([2.43774108e-03, 3.21216868e-01, 2.97331805e-01, 9.77450101e-01,
       1.41618989e-21, 4.03830330e-01, 1.41042759e-37])

In [ ]:
# show results
results = pd.DataFrame({'feature': np.r_[['Intercept'], 
                                         auto_df.drop(['mpg'],axis=1).columns.values],
                        'coefficients': beta,
                        'standard_error': SE_beta,
                        't-statistic': t,
                        'p-value': p})

results

,feature,coefficients,standard_error,t-statistic,p-value
0,Intercept,-14.535250,4.763882,-3.051136,2.437741e-03
1,cylinders,-0.329859,0.332104,-0.993240,3.212169e-01
2,displacement,0.007678,0.007358,1.043586,2.973318e-01
3,horsepower,-0.000391,0.013837,-0.028284,9.774501e-01
4,weight,-0.006795,0.000670,-10.140877,1.416190e-21
5,acceleration,0.085273,0.102036,0.835721,4.038303e-01
6,year,0.753367,0.052618,14.317630,1.410428e-37


คำตอบ: ตัวแปรที่มีค่า p-value < 0.05 ได้แก่ Intercept (beta0), weight, และ year

#### statsmodels library สามารถคำนวณค่าต่างๆ เหล่านี้ได้ ผลที่แสดงจะคล้ายกับใน R  
#### มีสองวิธึในการทำ linear regression: $\texttt{statsmodels.api.sm.OLS}$ 
#### และ $\texttt{statsmodels.formula.api.smf.ols}$

In [ ]:
# The statsmodels library provides a convenient means to get the
# same statistics
import statsmodels.api as sm

X = auto_df.drop(['mpg'], axis=1)
X = sm.add_constant(X)     # add bias constant
y = auto_df['mpg']

# add bias constant; 
#without this the equation turns into y = βx

# syntax is OLS(response, predictor)
model = sm.OLS(y, X).fit()
print(model.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Mon, 20 Dec 2021   Prob (F-statistic):          3.79e-135
Time:                        05:14:47   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -14.5353      4.764     -3.051   